In [360]:
# %pip install git+https://github.com/tensorflow/docs

import os.path

import numpy as np
import pandas as pd

from datetime import date
from thefuzz import fuzz

pd.options.mode.chained_assignment = None  # default='warn'
pd.options.display.max_colwidth = 160

from utils.configuration import settings
from utils.utils import *
from tft.api import *


In [361]:
API_KEY = settings.api_key
ASSETS_DIR = settings.assets_dir
SERVER = 'na1'  # euw1 na1 kr oc1
LEAGUE='challengers' # challengers grandmasters
MAX_COUNT = settings.max_count
LATEST_RELEASE = '12.12.450.4196' # '12.12.450.4196' Version 12.12.448.6653 12.11.446.9344
PATCH = date(2022, 7, 1)

TARGETNAME = 'placement'

In [362]:
summoners_df = pd.read_pickle(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_summoners.pickle'))

In [363]:
summoners_df

,id,accountId,puuid,name,profileIconId,revisionDate,summonerLevel,summonerId,summonerName,leaguePoints,rank,wins,losses,veteran,inactive,freshBlood,hotStreak
0,aM98OWZKwtQprZnmPfEQgyH__SScMb4ielz14Kbv49woQxQ,HLO_HWh7-0oHg0hgwWWuopqBLC4hh5_xsiFCowuQNoYBnjY,QmtvbmYjPxX6i5wBTRddnDen7XmbM6JAE84OcYF_eY3By_-i4yHe4vSJ_Kjxeiv9ArPXhlttPHeu4w,Jason Socks,5408,1657219691627,523,aM98OWZKwtQprZnmPfEQgyH__SScMb4ielz14Kbv49woQxQ,Jason Socks,502,I,100,69,False,False,True,False
1,0gkNDFyIZ1hZ6qZJcvQ5t8E-BQfTLeYJDAEPJ6cet1KgmI2d,JBjSFw_O3g_5QEgomj9AldeGzaPN6f4jaaUxbsrOtJbD-QkeD_Fo-aU-,IE7HMHPvvBmOicAHhwQ82QhDs-P64R15GDQ8Rg5fq9hEhlwnDWifj9qGHV_qgZHyfv3og1bZzcPD2w,VanillaXD,29,1657297937000,58,0gkNDFyIZ1hZ6qZJcvQ5t8E-BQfTLeYJDAEPJ6cet1KgmI2d,VanillaXD,533,I,187,157,False,False,True,False
2,6SiiTP1etDpYhnzK1IMb-D2RLoga1cTMCNJ1e4Q5gz5y9Co,CRfoIYZEQkHZ1EZqYay4h0VvhIfirU60fTKSaut96j7O1A,SUpb3Wr5I1GPOU48IqRdeeXDkr9Vcxr0pdahQ6t05EWdX_dnuEdX2yK0HdEAqFdAZSjSGiWkiLBO3Q,Trugz,5181,1657292366000,182,6SiiTP1etDpYhnzK1IMb-D2RLoga1cTMCNJ1e4Q5gz5y9Co,Trugz,505,I,118,81,False,False,True,True
3,NZUyBA6Bjiy_MZj8ShTluVnWxVfPA3NiKmzGuOCH2zNPkhM,sVsaODH09TvWBNVl4uLjnx4rHJ1DeAGAdUFf8LNIMj8jym4,dZKtDFv3NazDhbfzEYQh3Lz67zQfRJFblxCWcI6ZrSh6ArecuX7mZPL6JKGnKaZOqY1Z4Y4dVGH61g,ajmonsta,1591,1657297789000,399,NZUyBA6Bjiy_MZj8ShTluVnWxVfPA3NiKmzGuOCH2zNPkhM,ajmonsta,825,I,169,135,False,False,True,False
4,5dE6M7nXMdLATafB_x0kKwMQ0AVLZrLrt3oUnVrgmQRMHiU,gaVXvnE4upEw9z_3e-DMx9GyhIKxk54EUhiPu68kH9JeYgo,YG4qyuxricddSWKg4g0vF410Adn7lMFm2BayIF2a2BxrOVjXwKGpw9TQxuLVIm5S-LaAMAghpTnPOg,garet,0,1657286217000,210,5dE6M7nXMdLATafB_x0kKwMQ0AVLZrLrt3oUnVrgmQRMHiU,garet,1304,I,168,93,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,PwY95sJrHM_yUyedet6IyPUuqX5G_vVrcMYtdtAJ8_dPhIM,n0DwVbP6PAxH4Q0yIt4FE2kQZprEZJKeEaWytmkdUjQJqek,GNZgsn4Ja072f0o99inDwqCdwjgtU8-saoh1B6yyoGUnOsykCsM8-yKTzc-4mXFrhFOqac91a7LslQ,ECohmygod,7,1657221179000,34,PwY95sJrHM_yUyedet6IyPUuqX5G_vVrcMYtdtAJ8_dPhIM,ECohmygod,533,I,212,187,False,False,True,False
201,e34rJaogiO4CE8zLS8ewLMFGPGoNIMIMsB7gDqVH9aPNHIuY,BcLHPFVnuMFT_g6cJdB7LsDocnWKJ3TQyp9mMIY7amMcq-PdOm3O0410,St_wL0UOxf_11zObuZn-7FE-yGRGjfjFaeHlAbCMCESgyqgS636iSJhu5qxxDqBq7XuNlL4I161r8g,Spethom,4904,1657297246826,198,e34rJaogiO4CE8zLS8ewLMFGPGoNIMIMsB7gDqVH9aPNHIuY,Spethom,610,I,143,93,False,False,False,False
202,_qohVpjRSfy3xKoBpELkQaiVr0hnBq6Q7cVCOU1wYFE-tUyf,MaTrdhM5ZFuPPjYtfpng-SwqCteoIyGviwsAnA3f5G7LCwaPHTbASRG1,7BoKQcZCku1BMuaDQbXvwMkNWKBn9frOYViJTUbhQld7ZaOu0mpSggbyThbbE1jCcyf7fWemHvRHMg,cantkilltrag,14,1657258042000,18,_qohVpjRSfy3xKoBpELkQaiVr0hnBq6Q7cVCOU1wYFE-tUyf,cantkilltrag,504,I,144,111,False,False,True,False
203,f7_bi4CEGkGEPHpfMZkVU7zAz-XV4vl8QWhqiDMkRs3GUH4,SH_RhxMDbtdcSkevgJitMUaZiquawNSIFa0wDWa5DRArFxs,01S6NdFXqrUTGRZWrB31C-RlV52Szh-jlZvBx4SGqEX2z3XDzWfc7zg18i9bxgmaUob7xvpX61eRyA,sehm,3588,1657264401628,577,f7_bi4CEGkGEPHpfMZkVU7zAz-XV4vl8QWhqiDMkRs3GUH4,sehm,623,I,129,99,False,False,True,True


In [364]:

# Get all unique matches_id from assets dir
matches_asset = load_matches(summoners_df, server=SERVER)
matches_id = [match['metadata']['match_id'] for match in matches_asset]
seen = set()
uniq_matches_id = [
    x for x in matches_id if x not in seen and not seen.add(x)]

In [365]:
len(uniq_matches_id)

6961

In [366]:
len(matches_asset)

11112

In [367]:
seen = set()
seen_add = seen.add
uniq_matches = [x for x in matches_asset if x['metadata']['match_id'] not in seen and not seen_add(x['metadata']['match_id'])]

In [368]:
len(uniq_matches)

6961

In [369]:
matches_asset[300]['info']['game_version']

'Version 12.11.446.9344 (Jun 09 2022/13:23:39) [PUBLIC] <Releases/12.11>'

In [370]:
latest_matches = [match for match in uniq_matches if (
    LATEST_RELEASE in match['info']['game_version'])]


In [371]:
latest_patch_matches = [match for match in uniq_matches if (LATEST_RELEASE in match['info']['game_version'])
                        and (PATCH <= date.fromtimestamp(match['info']['game_datetime']/1000.0))]


In [372]:
len(latest_matches)

5068

In [373]:
len(latest_patch_matches)

3009

In [374]:
# latest_matches[0]['info']

In [375]:
from typing import List


def process_matches(df) -> List:
    matches_array = []

    for match_row in df:
        match_id = match_row['metadata']['match_id']

        for participant in match_row['info']['participants']:
            match = {}
            match['match_id'] = match_id
            # match['level'] = participant['level']
            match['placement'] = participant['placement']
            # match['players_eliminated'] = participant['players_eliminated']
            # match['total_damage_to_players'] = participant['total_damage_to_players']

            for augment_index, augment in enumerate(participant['augments']):
                # if augment == 'TFT7_Augment_GuildLootHR':
                #     augment = 'TFT7_Augment_BandOfThieves1'
                match[f'augment{augment_index}'] = augment

            for trait_index, trait in enumerate(participant['traits']):
                match[f'{trait["name"]}'] = trait["tier_current"]

            for unit_index, unit in enumerate(participant['units']):
                match[f'{unit["character_id"]}'] = unit["tier"]
                match['TFT7_TrainerDragon_item1'] = 'None'
                match['TFT7_TrainerDragon_item2'] = 'None'
                for item_index, item in enumerate(unit['itemNames']):
                    match[f'{unit["character_id"]}_item{item_index}'] = item.split('_')[-1]

            matches_array.append(match)

    return matches_array


In [376]:
matches_array = process_matches(latest_matches)
matches_patch_array = process_matches(latest_patch_matches)

In [377]:
matches_array[0]
# len(matches_array)

{'match_id': 'NA1_4365110158',
 'placement': 2,
 'augment0': 'TFT6_Augment_TrueTwos',
 'augment1': 'TFT6_Augment_Recombobulator',
 'augment2': 'TFT7_Augment_AxiomArc2',
 'Set7_Assassin': 1,
 'Set7_Bruiser': 2,
 'Set7_Dragon': 1,
 'Set7_Guild': 1,
 'Set7_Legend': 0,
 'Set7_Mage': 0,
 'Set7_Ragewing': 0,
 'Set7_Tempest': 1,
 'Set7_Warrior': 0,
 'Set7_Whispers': 2,
 'TFT7_Qiyana': 2,
 'TFT7_TrainerDragon_item1': 'None',
 'TFT7_TrainerDragon_item2': 'None',
 'TFT7_Shen': 2,
 'TFT7_Sylas': 2,
 'TFT7_Sylas_item0': 'Shroud',
 'TFT7_Sylas_item1': 'RedBuff',
 'TFT7_Sylas_item2': 'DragonsClaw',
 'TFT7_Talon': 2,
 'TFT7_DragonPurple': 3,
 'TFT7_DragonPurple_item0': 'InfinityEdge',
 'TFT7_DragonPurple_item1': 'RapidFireCannon',
 'TFT7_DragonPurple_item2': 'Bloodthirster',
 'TFT7_Ornn': 1,
 'TFT7_Ornn_item0': 'SeraphsEmbrace',
 'TFT7_Ornn_item1': 'Redemption',
 'TFT7_Ornn_item2': 'TitansResolve',
 'TFT7_Pyke': 1,
 'TFT7_Pyke_item0': 'ThiefsGloves',
 'TFT7_Pyke_item1': 'WarmogsArmor',
 'TFT7_Pyke_it

In [378]:
matches_league_df = pd.json_normalize(matches_array)
matches_league_patch_df = pd.json_normalize(matches_patch_array)

In [379]:
matches_league_df

,match_id,placement,augment0,augment1,augment2,Set7_Assassin,Set7_Bruiser,Set7_Dragon,Set7_Guild,Set7_Legend,...,TFT7_Vladimir_item0,TFT7_Vladimir_item1,TFT7_Vladimir_item2,TFT7_Shen_item2,TFT7_Thresh_item1,TFT7_Thresh_item2,TFT7_Taric_item1,TFT7_Heimerdinger_item1,TFT7_Heimerdinger_item2,TFT7_Taric_item2
0,NA1_4365110158,2,TFT6_Augment_TrueTwos,TFT6_Augment_Recombobulator,TFT7_Augment_AxiomArc2,1.0,2.0,1.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NA1_4365110158,1,TFT7_Augment_BruiserEmblem,TFT6_Augment_CyberneticShell1,TFT6_Augment_CelestialBlessing2,NaN,1.0,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NA1_4365110158,8,TFT6_Augment_Ascension,TFT6_Augment_PandorasItems,TFT7_Augment_MageEmblem,NaN,0.0,NaN,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NA1_4365110158,7,TFT7_Augment_AxiomArc2,TFT6_Augment_PandorasItems,TFT7_Augment_RagewingTantrum,0.0,0.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NA1_4365110158,5,TFT7_Augment_TempestEmblem,TFT7_Augment_AxiomArc1,TFT6_Augment_PortableForge,0.0,0.0,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40539,NA1_4358279197,5,TFT6_Augment_MetabolicAccelerator,TFT6_Augment_ForceOfNature,TFT6_Augment_BlueBattery2,NaN,2.0,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40540,NA1_4358279197,6,TFT6_Augment_CelestialBlessing2,TFT6_Augment_MaxLevel10,TFT6_Augment_SecondWind2,1.0,1.0,NaN,3.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40541,NA1_4358279197,8,TFT7_Augment_MirageEmblem,TFT6_Augment_LudensEcho3,TFT7_Augment_UrfsGrabBag1,1.0,NaN,1.0,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40542,NA1_4358279197,1,TFT6_Augment_CyberneticUplink2,TFT6_Augment_CelestialBlessing3,TFT6_Augment_CyberneticImplants2,0.0,2.0,1.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [380]:
matches_league_df[matches_league_df.isnull().T.any()]

,match_id,placement,augment0,augment1,augment2,Set7_Assassin,Set7_Bruiser,Set7_Dragon,Set7_Guild,Set7_Legend,...,TFT7_Vladimir_item0,TFT7_Vladimir_item1,TFT7_Vladimir_item2,TFT7_Shen_item2,TFT7_Thresh_item1,TFT7_Thresh_item2,TFT7_Taric_item1,TFT7_Heimerdinger_item1,TFT7_Heimerdinger_item2,TFT7_Taric_item2
0,NA1_4365110158,2,TFT6_Augment_TrueTwos,TFT6_Augment_Recombobulator,TFT7_Augment_AxiomArc2,1.0,2.0,1.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NA1_4365110158,1,TFT7_Augment_BruiserEmblem,TFT6_Augment_CyberneticShell1,TFT6_Augment_CelestialBlessing2,NaN,1.0,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NA1_4365110158,8,TFT6_Augment_Ascension,TFT6_Augment_PandorasItems,TFT7_Augment_MageEmblem,NaN,0.0,NaN,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NA1_4365110158,7,TFT7_Augment_AxiomArc2,TFT6_Augment_PandorasItems,TFT7_Augment_RagewingTantrum,0.0,0.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NA1_4365110158,5,TFT7_Augment_TempestEmblem,TFT7_Augment_AxiomArc1,TFT6_Augment_PortableForge,0.0,0.0,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40539,NA1_4358279197,5,TFT6_Augment_MetabolicAccelerator,TFT6_Augment_ForceOfNature,TFT6_Augment_BlueBattery2,NaN,2.0,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40540,NA1_4358279197,6,TFT6_Augment_CelestialBlessing2,TFT6_Augment_MaxLevel10,TFT6_Augment_SecondWind2,1.0,1.0,NaN,3.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40541,NA1_4358279197,8,TFT7_Augment_MirageEmblem,TFT6_Augment_LudensEcho3,TFT7_Augment_UrfsGrabBag1,1.0,NaN,1.0,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40542,NA1_4358279197,1,TFT6_Augment_CyberneticUplink2,TFT6_Augment_CelestialBlessing3,TFT6_Augment_CyberneticImplants2,0.0,2.0,1.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Sort and reorder columns

In [381]:
def reorder_df_col(df):
    fixed_cols = ['placement', 'match_id', 'augment0', 'augment1', 'augment2']
    all_cols = df.columns
    to_sort_cols = list(set(all_cols) - set(fixed_cols))

    return df.reindex(columns=fixed_cols + sorted(to_sort_cols))


In [382]:
matches_league_df = reorder_df_col(matches_league_df)
matches_league_patch_df = reorder_df_col(matches_league_patch_df)

# Output dataframes

In [383]:
matches_league_df.to_pickle(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_matches.pickle'))
matches_league_df.to_csv(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_matches.csv'), index=False)
matches_league_patch_df.to_pickle(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_{PATCH}_matches.pickle'))
matches_league_patch_df.to_csv(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_{PATCH}_matches.csv'), index=False)

# End